In [1]:
%matplotlib inline

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
import twstock as ts
import time 
import threading
import queue
import json
from urllib.request import urlopen

In [265]:
year='2018'
month="10"

dpc=pd.read_csv(f"C:/Users/黃柏倫/Desktop/data/{month}pc.csv",encoding = "big5")
dpc.iloc[:,1:]=np.array(dpc.iloc[:,:-1])

dpc.交易日期=dpc.index
dpc.index=range(len(dpc.index))

In [266]:
dpcW1=dpc[(dpc['到期月份(週別)']==f'{year}{month}W1')]
dpcW1.index=range(len(dpcW1.index))
dpcW1=dpcW1.replace('-',0)

In [267]:
dpcW2=dpc[(dpc['到期月份(週別)']==f'{year}{month}W2')]
dpcW2.index=range(len(dpcW2.index))
dpcW2=dpcW2.replace('-',0)
#dpcW2[(dpcW2['履約價']==10000.0)]&(dpcW2['交易時段']=='一般')

In [268]:
dpcW3=dpc[(dpc['到期月份(週別)']==f'{year}{month}  ')]
dpcW3.index=range(len(dpcW3.index))
dpcW3=dpcW3.replace('-',0)

In [269]:
df=pd.read_csv(f"C:/Users/黃柏倫/Desktop/data/{month}f.csv",encoding = "big5")
df.iloc[:,1:]=np.array(df.iloc[:,:-1])
df.交易日期=df.index
df.index=range(len(df.index))

In [270]:
df=df[(df['到期月份(週別)']==f'{year}{month}  ')]#&(df['交易時段']=='一般')
df.index=range(len(df.index))
df.開盤價=df.開盤價.astype(float)
df.最高價=df.最高價.astype(float)
df.最低價=df.最低價.astype(float)
df.收盤價=df.收盤價.astype(float)

In [279]:
def get(s,cd,i):
    return float(cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]
            [['開盤價']].astype(float).sum()[0])

In [280]:
def getpc(s,cd,i):
    if (list(cd[(cd['履約價']==s+25)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'])==[]):        
        s=int(round(df.loc[i,'開盤價']/100)*100)
        c=float(cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'].iloc[0])
    else:
        c=float(cd[(cd['履約價']==s+25)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'].iloc[0])
    
    if (list(cd[(cd['履約價']==s-25)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'])==[]):        
        s=int(round(df.loc[i,'開盤價']/100)*100)
        p=float(cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'].iloc[1])
    else:
        p=float(cd[(cd['履約價']==s-25)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'].iloc[1])
    
    return c+p

In [281]:
def buy(s,cd,i):
    return float(cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])
              &(cd['交易時段']==df.loc[i,'交易時段'])]['收盤價'].astype(float).sum())  

In [282]:
def buypc(s,cd,i):
    if (list(cd[(cd['履約價']==s+25)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'])==[]):       
        s=int(round(df.loc[i,'收盤價']/100)*100)
        c=float(cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['收盤價'].iloc[0])
    else:
        c=float(cd[(cd['履約價']==s+25)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['收盤價'].iloc[0])
    if (list(cd[(cd['履約價']==s-25)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'])==[]):
        s=int(round(df.loc[i,'收盤價']/100)*100)
        p=float(cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['收盤價'].iloc[1])
    else:
        p=float(cd[(cd['履約價']==s-25)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['收盤價'].iloc[1])
   
    return c+p

In [283]:
def rundetail(m=0,r=25,dif=55,d=len(df.index)):
    profit=0
    h=0

    for i in range(d):

        s=int(round(df.loc[i,'開盤價']/r)*r)

        for cd in [dpcW1,dpcW2,dpcW3]:
            cd.loc[len(cd.index)-1,'交易日期']
            if df.loc[i,'交易日期']<=cd.loc[len(cd.index)-1,'交易日期']:
                break
        s=s-m
        if i==0:

            s0=s

        if (h==1)&(abs(s0-s)>dif):
            if (s/25)%2==0:
                if (s0/25)%2==0:
                    get0m=get(s0,cd,i)
                    getm=get(s,cd,i)
                    profit=profit-get0m+getm       
                    print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},轉倉 點位{s0}價格{get0m}-->點位{s}價格{getm}")
                else:
                    get0m=getpc(s0,cd,i)
                    getm=get(s,cd,i)            
                    profit=profit-get0m+getm                  
                    print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},轉倉 點位{s0+25}Call和{s0-25}Put價格{get0m}-->點位{s}價格{getm}")        
            if (s/25)%2==1:
                if (s0/25)%2==0:
                    get0m=get(s0,cd,i)
                    getm=getpc(s,cd,i)  
                    profit=profit-get0m+getm            
                    print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},轉倉 點位{s0}價格{get0m}-->點位{s+25}Call和{s-25}Put價格{getm}")

                else:
                    get0m=getpc(s0,cd,i)
                    getm=getpc(s,cd,i)  
                    profit=profit-get0m+getm            
                    print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},轉倉 點位{s0+25}Call和{s0-25}Put價格{get0m}-->點位{s+25}Call和{s-25}Put價格{getm}")
            s0=s
        elif h==0:
            if (s/25)%2==0:
                getm=get(s,cd,i)
                profit=profit+getm
                h=1
                print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},進場 點位{s}價格{getm}")

            if (s/25)%2==1:
                getm=getpc(s,cd,i)

                profit=profit+getm
                h=1
                print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},進場 點位{s+25}Call和{s-25}Put價格{getm}")
            s0=s
        else:
            print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},沒動作")


        if (h==1)&(df.loc[i,'交易日期']==cd.loc[len(cd.index)-1,'交易日期'])&(df.loc[i,'交易時段']=='盤後'):
            if (s0/25)%2==0:
                buym=buy(s0,cd,i)
                h=0
                profit=profit-buym
                print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},買回 點位{s0}價格{buym}")
                

            if (s0/25)%2==1:
                buym=buypc(s0,cd,i)
                h=0
                profit=profit-buym
                print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},點位選{s},期貨{int(df.loc[i,'開盤價'])},買回 點位{s0+25}Call和{s0-25}Put價格{buym}")
        
    print(f"手中{h}組,價值{profit}")
    return profit
    


In [284]:
def run(m=0,r=25,dif=55,d=len(df.index)):
    profit=0
    h=0

    for i in range(d):


        s=int(round(df.loc[i,'開盤價']/r)*r)
#        print(f"點位選{s}")
        for cd in [dpcW1,dpcW2,dpcW3]:
            cd.loc[len(cd.index)-1,'交易日期']
            if df.loc[i,'交易日期']<=cd.loc[len(cd.index)-1,'交易日期']:
                break
        s=s-m
        if i==0:

            s0=s

        if (h==1)&(abs(s0-s)>dif):
            if (s/25)%2==0:
                if (s0/25)%2==0:
                    get0m=get(s0,cd,i)
                    getm=get(s,cd,i)
                    profit=profit-get0m+getm       
#                    print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},轉倉 點位{s0}價格{get0m}-->點位{s}價格{getm}")
                else:
                    get0m=getpc(s0,cd,i)
                    getm=get(s,cd,i)            
                    profit=profit-get0m+getm                  
#                    print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},轉倉 點位{s0+25}Call和{s0-25}Put價格{get0m}-->點位{s}價格{getm}")        
            if (s/25)%2==1:
                if (s0/25)%2==0:
                    get0m=get(s0,cd,i)
                    getm=getpc(s,cd,i)  
                    profit=profit-get0m+getm            
#                    print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},轉倉 點位{s0}價格{get0m}-->點位{s+25}Call和{s-25}Put價格{getm}")

                else:
                    get0m=getpc(s0,cd,i)
                    getm=getpc(s,cd,i)  
                    profit=profit-get0m+getm            
#                    print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},轉倉 點位{s0+25}Call和{s0-25}Put價格{get0m}-->點位{s+25}Call和{s-25}Put價格{getm}")
            s0=s
        elif h==0:
            if (s/25)%2==0:
                getm=get(s,cd,i)
                profit=profit+getm
                h=1
#                print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},進場 點位{s}價格{getm}")

            if (s/25)%2==1:
                getm=getpc(s,cd,i)

                profit=profit+getm
                h=1
#                print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},進場 點位{s+25}Call和{s-25}Put價格{getm}")
            s0=s
#        else:
#            print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},沒動作")


        if (h==1)&(df.loc[i,'交易日期']==cd.loc[len(cd.index)-1,'交易日期'])&(df.loc[i,'交易時段']=='盤後'):
            if (s0/25)%2==0:
                buym=buy(s0,cd,i)
                h=0
                profit=profit-buym
#                print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},買回 點位{s0}價格{buym}")
                

            if (s0/25)%2==1:
                buym=buypc(s0,cd,i)
                h=0
                profit=profit-buym
#               print(f"{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},期貨{int(df.loc[i,'開盤價'])},買回 點位{s0+25}Call和{s-25}Put價格{buym}")
            
#   print(f"手中{h}組,價值{profit}")
    return profit
    


In [285]:
m=0
r=25
d=len(df.index)
for dif in np.arange(0,501,50):
    print(f"價差{dif},獲益{run(m,r,dif,d)}")

價差0,獲益-468.80000000000007
價差50,獲益-275.09999999999997
價差100,獲益-208.10000000000002
價差150,獲益-107.60000000000002
價差200,獲益-208.89999999999995
價差250,獲益-350.29999999999995
價差300,獲益-350.29999999999995
價差350,獲益-346.2
價差400,獲益-547.5
價差450,獲益-547.5
價差500,獲益-547.5


In [286]:
m=0
r=25
dif=150
d=len(df.index)
rundetail(m,r,dif,d)

2018/10/01一般,點位選10975,期貨10968,進場 點位11000Call和10950Put價格76.5
2018/10/01盤後,點位選10950,期貨10943,沒動作
2018/10/02一般,點位選11000,期貨11006,沒動作
2018/10/02盤後,點位選11000,期貨11008,沒動作
2018/10/03一般,點位選10875,期貨10880,沒動作
2018/10/03盤後,點位選10875,期貨10863,沒動作
2018/10/03盤後,點位選10875,期貨10863,買回 點位11000Call和10950Put價格54.9
2018/10/04一般,點位選10825,期貨10823,進場 點位10850Call和10800Put價格105.5
2018/10/04盤後,點位選10875,期貨10864,沒動作
2018/10/05一般,點位選10650,期貨10658,轉倉 點位10850Call和10800Put價格156.5-->點位10650價格136.0
2018/10/05盤後,點位選10675,期貨10680,沒動作
2018/10/08一般,點位選10475,期貨10470,轉倉 點位10650價格194.5-->點位10500Call和10450Put價格127.0
2018/10/08盤後,點位選10525,期貨10513,沒動作
2018/10/09一般,點位選10500,期貨10491,沒動作
2018/10/09盤後,點位選10450,期貨10441,沒動作
2018/10/11一般,點位選10025,期貨10026,轉倉 點位10500Call和10450Put價格302.2-->點位10050Call和10000Put價格200.0
2018/10/11盤後,點位選10450,期貨10460,轉倉 點位10050Call和10000Put價格86.0-->點位10450價格87.0
2018/10/11盤後,點位選10450,期貨10460,買回 點位10450價格72.0
2018/10/12一般,點位選9725,期貨9736,進場 點位9750Call和9700Put價格268.0
2018/10/12盤後,點位選9700,期貨9705,沒動作
2018/10/15一般,點位選9900

-107.60000000000002

In [228]:
cd=dpcW2
mor='一般'
nig='盤後'
cd[(cd['履約價']==10850.0)&(cd['交易時段']==mor)&(cd['交易日期']=='2018/10/05')].head(2)

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段


In [198]:
cd[(cd['履約價']==10400.0)&(cd['交易時段']==mor)&(cd['交易日期']=='2018/10/05')].head(2)

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段
186,2018/10/05,TXO,201810W2,10800.0,買權,16,21,1.1,3.8,33796,3.8,14318,3.7,3.8,146,1.1,NaN,一般
188,2018/10/05,TXO,201810W2,10800.0,賣權,147,378,131,283,3819,283,2910,282,302,378,26,NaN,一般


In [199]:
cd[(cd['履約價']==10650.0)&(cd['交易時段']==mor)&(cd['交易日期']=='2018/10/05')].head(2)

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段
174,2018/10/05,TXO,201810W2,10650.0,買權,73,87,9.4,20.5,52469,20.5,8601,20.5,21,247,9.4,NaN,一般
176,2018/10/05,TXO,201810W2,10650.0,賣權,63,206,46,152,42200,152,6732,152,163,206,7.5,NaN,一般
